In [ ]:
import os
import re
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
from collections import namedtuple
import numpy as np
import pandas as pd
from typing import *
from tqdm.notebook import tqdm
from sklearn.utils.extmath import softmax
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
import gc
import random

import torch

In [ ]:
pd.set_option('display.max_column', None)

In [ ]:
import sys
sys.path = [
    '../usr/lib/script_jigsaw_rbase_ridge_chkpt_fold_seeds',
    '../usr/lib/script_jigsaw_roberta_base_chkpt_fold_seeds',
    '../usr/lib/script_jigsaw_rbase_ridge_chkpt_agg_fold_seeds',
    '../usr/lib/script_jigsaw_rbase_chkpt_agg_fold_seeds',
    
    '../usr/lib/script_jigsaw_rbase_ridge_ce_chkpt_fold_seeds',
    '../usr/lib/script_jigsaw_rbase_ce_chkpt_fold_seeds',
    '../usr/lib/script_jigsaw_rbase_ce_chkpt_agg_fold_seeds',
    '../usr/lib/script_jigsaw_rbase_ridge_ce_chkpt_agg_fold_seeds',
    
    '../usr/lib/script_jigsaw_rbase_triplet_fold_seeds',
    '../usr/lib/script_jigsaw_rbase_ridge_triplet_agg_folds',
    '../usr/lib/script_jigsaw_rbase_ridge_triplet_fold_seeds',
    '../usr/lib/script_jigsaw_rbase_triplet_agg_fold_seeds',
    
    '../usr/lib/two_much_overfitting',

] + sys.path

In [ ]:
import script_jigsaw_rbase_ridge_chkpt_fold_seeds
import script_jigsaw_rbase_chkpt_fold_seeds
import script_jigsaw_rbase_ridge_chkpt_agg_fold_seeds
import script_jigsaw_rbase_chkpt_agg_fold_seeds


import script_jigsaw_rbase_ridge_ce_chkpt_fold_seeds
import script_jigsaw_rbase_ce_chkpt_fold_seeds
import script_jigsaw_rbase_ce_chkpt_agg_fold_seeds
import script_jigsaw_rbase_ridge_ce_chkpt_agg_fold_seeds


import script_jigsaw_rbase_triplet_fold_seeds
import script_jigsaw_rbase_ridge_triplet_agg_folds
import script_jigsaw_rbase_ridge_triplet_fold_seeds
import script_jigsaw_rbase_triplet_agg_fold_seeds

import two_much_overfitting

In [ ]:
preds = []

# Overfit Ridge

In [ ]:
sub_overfit = two_much_overfitting.data_with_overfitting_issues()

# Triplet Models

In [ ]:
sub_rbase_tr = script_jigsaw_rbase_triplet_fold_seeds.generate_rankings()
preds.append(sub_rbase_tr)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
sub_rbase_ridge_tr_agg = script_jigsaw_rbase_ridge_triplet_agg_folds.generate_rankings()
preds.append(sub_rbase_ridge_tr_agg)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
sub_rbase_ridge_tr = script_jigsaw_rbase_ridge_triplet_fold_seeds.generate_rankings()
preds.append(sub_rbase_ridge_tr)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
sub_rbase_tr_agg = script_jigsaw_rbase_triplet_agg_fold_seeds.generate_rankings()
preds.append(sub_rbase_ridge_tr_agg)

torch.cuda.empty_cache()
gc.collect()

## Agg Models

In [ ]:
sub_rbase_ce_agg = script_jigsaw_rbase_ce_chkpt_agg_fold_seeds.generate_rankings()
preds.append(sub_rbase_ce_agg)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
sub_rbase_agg = script_jigsaw_rbase_chkpt_agg_fold_seeds.generate_rankings()
preds.append(sub_rbase_agg)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
sub_rbase_ridge_ce_agg = script_jigsaw_rbase_ridge_ce_chkpt_agg_fold_seeds.generate_rankings()
preds.append(sub_rbase_ridge_ce_agg)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
sub_rbase_ridge_agg = script_jigsaw_rbase_ridge_chkpt_agg_fold_seeds.generate_rankings()
preds.append(sub_rbase_ridge_agg)

torch.cuda.empty_cache()
gc.collect()

## Full Data Models

In [ ]:
sub_rbase_ce = script_jigsaw_rbase_ce_chkpt_fold_seeds.generate_rankings()
preds.append(sub_rbase_ce)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
sub_rbase_ce_ridge = script_jigsaw_rbase_ridge_ce_chkpt_fold_seeds.generate_rankings()
preds.append(sub_rbase_ce_ridge)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
sub_rbase_ridge = script_jigsaw_rbase_ridge_chkpt_fold_seeds.generate_rankings()
preds.append(sub_rbase_ridge)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
sub_rbase = script_jigsaw_rbase_chkpt_fold_seeds.generate_rankings()
preds.append(sub_rbase)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
df = pd.DataFrame()

for i, rdf in enumerate(preds):
    for j, c in enumerate(rdf.columns):
        df.loc[:, f'pred_{i}_{j}'] = rdf.loc[:, c].values
   

In [ ]:
df.head()

In [ ]:
df['pred'] = df.mean(axis=1)
df['pred'] = df['pred'].rank(method='first')

final = df['pred'].values

In [ ]:
w = 0.5
ens = w*final + (1 - w)*sub_overfit.score.values

In [ ]:
sub = pd.read_csv('../input/jigsaw-toxic-severity-rating/sample_submission.csv')
sub['score'] = ens
sub['score'] = sub['score'].rank(method='first')
sub.to_csv('submission.csv', index=False)